In [126]:
# Importing requiered libraries
import ast
import pandas as pd
import gzip
# import matplotlib.pyplot as plt
import importlib
import data_analysis_utils

# Reload the functions module
importlib.reload(data_analysis_utils)

<module 'data_analysis_utils' from 's:\\Courses\\Data Science\\0. Henry\\M7 - Labs - Project MLOps _ steam\\game-recommendation-system\\eda\\data_analysis_utils.py'>

# User Reviews

## Importing file

In [127]:
# Importing user_reviews.csv file
user_reviews_path = "datasets/user_reviews.csv"

# Setting low_memory=False - To read the entire file so it determines the data types accurately. 
# Note: It can increase memory usage for larger datasets
user_reviews_1 = pd.read_csv(user_reviews_path, low_memory=False)

## Data Preparation & Transformation

The current data 

In [128]:
user_reviews_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25485 entries, 0 to 25484
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      25485 non-null  object
 1   user_url     25485 non-null  object
 2   item_id      25485 non-null  object
 3   posted       25485 non-null  object
 4   last_edited  25485 non-null  object
 5   helpful      25485 non-null  object
 6   recommend    25485 non-null  object
 7   funny        25485 non-null  object
 8   review       25485 non-null  object
dtypes: object(9)
memory usage: 1.8+ MB


In [129]:
# converting values to it's corresponding data types
user_reviews_2 = user_reviews_1.copy()
user_reviews_2['item_id'] = user_reviews_2['item_id'].apply(ast.literal_eval)
user_reviews_2['posted'] = user_reviews_2['posted'].apply(ast.literal_eval)
user_reviews_2['last_edited'] = user_reviews_2['last_edited'].apply(ast.literal_eval)
user_reviews_2['recommend'] = user_reviews_2['recommend'].apply(ast.literal_eval)
user_reviews_2['review'] = user_reviews_2['review'].apply(ast.literal_eval)

Sometimes user names can be the same between user.<br>
Just to verify that the current user IDs are unique, and no user names.<br>
The user IDs will be compared with user_url, duw to user_url is unique for each user.

In [130]:
# Iteration of the rows and compare them, if user IDs don't match it will print the index and the value
for index, row in user_reviews_2.iterrows():
    id_value = row["user_id"]
    url_value = row["user_url"]
    
    id_len = len(id_value)
    url_id = url_value[-id_len:]
    if id_value != url_id:
        print(index, id_value)

### Column "posted"

During data preparation, some dates couldn't be converted to "yyyy-mm-dd", those dates will be compared with the values from "last_edited" column.

In [131]:
# Verifying data type of the column
posted_data_type = data_analysis_utils.column_data_types_summary(user_reviews_2, "posted")
posted_data_type

{'column_name': 'posted', 'data_type_counts': {"<class 'list'>": 25485}}

In [132]:
# Verifying data type of the column
posted_data_type = data_analysis_utils.column_data_types_summary(user_reviews_2, "last_edited")
posted_data_type

{'column_name': 'last_edited', 'data_type_counts': {"<class 'list'>": 25485}}

In [133]:
# Creating a copy of the dataframe
user_reviews_3 = user_reviews_2.copy()

# The "Posted %B %d." patter of the datetime values of the datframe will automatically take the year 1900,
# this will help to avoid removing date from the dataframe

# Setting the original datetime patterns
original_date_list = ["Posted %B %d, %Y.", "Posted %B %d.", "%Y-%m-%d"]

# Transforming to datetime type using "convert_column_to_dates" function
data_analysis_utils.convert_column_to_dates(user_reviews_3, "posted", original_date_list)

{'total_rows': 25485,
 'column_name': 'posted',
 'num_failed_conversions': 24,
 'report': {348: {6: 'Posted February 29.'},
  1544: {1: 'Posted February 29.'},
  2696: {3: 'Posted February 29.'},
  2905: {2: 'Posted February 29.'},
  3852: {3: 'Posted February 29.'},
  5061: {0: 'Posted February 29.'},
  6037: {0: 'Posted February 29.'},
  6104: {0: 'Posted February 29.'},
  9111: {0: 'Posted February 29.'},
  10454: {1: 'Posted February 29.'},
  10493: {0: 'Posted February 29.'},
  10633: {0: 'Posted February 29.'},
  10651: {0: 'Posted February 29.'},
  11404: {1: 'Posted February 29.'},
  11945: {0: 'Posted February 29.'},
  14914: {1: 'Posted February 29.'},
  15554: {2: 'Posted February 29.'},
  16112: {0: 'Posted February 29.'},
  16143: {0: 'Posted February 29.'},
  16439: {0: 'Posted February 29.'},
  20438: {0: 'Posted February 29.', 1: 'Posted February 29.'},
  21898: {0: 'Posted February 29.'},
  24150: {4: 'Posted February 29.'},
  24457: {0: 'Posted February 29.'}}}

In [134]:
print(user_reviews_3['posted'][2696])
print(user_reviews_3['last_edited'][2696])
print(user_reviews_3['recommend'][2696])

['1900-05-20', '1900-07-14', '1900-03-07', 'Posted February 29.', '1900-02-10', '1900-02-09', '1900-02-08']
['', '', '', '', '', '', '']
[True, True, True, True, True, True, True]


## EDA

### Summary Statistics

### Data Visualization

### Correlation Analysis

### Feature Relationships

### Outlier Detection

### Text Analysis

### Documentation and Reporting

Sentiment analysis of review

# User Items

## Importing file

In [135]:
# Importing user_items.csv file
user_items_path = "datasets/user_items.csv"

# Setting low_memory=False - To read the entire file so it determines the data types accurately. 
# Note: It can increase memory usage for larger datasets
user_items_1 = pd.read_csv(user_items_path, low_memory=False)

## Data Preparation & Transformation

In [136]:
user_items_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87625 entries, 0 to 87624
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           87625 non-null  object
 1   steam_id          87625 non-null  int64 
 2   user_url          87625 non-null  object
 3   items_count       87625 non-null  int64 
 4   item_id           87625 non-null  object
 5   item_name         87625 non-null  object
 6   playtime_forever  87625 non-null  object
 7   playtime_2weeks   87625 non-null  object
dtypes: int64(2), object(6)
memory usage: 5.3+ MB


In [137]:
# converting values to it's corresponding data types
user_items_2 = user_items_1.copy()
user_items_2['item_id'] = user_items_2['item_id'].apply(ast.literal_eval)
user_items_2['item_name'] = user_items_2['item_name'].apply(ast.literal_eval)
user_items_2['playtime_forever'] = user_items_2['playtime_forever'].apply(ast.literal_eval)

Sometimes user names can be the same between user.<br>
Just to verify that the current user IDs are unique, and no user names.<br>
The user IDs will be compared with user_url, duw to user_url is unique for each user.

In [138]:
# Iteration of the rows and compare them, if user IDs don't match it will print the index and the value
for index, row in user_items_2.iterrows():
    id_value = row["user_id"]
    url_value = row["user_url"]
    
    id_len = len(id_value)
    url_id = url_value[-id_len:]
    if id_value != url_id:
        print(index, id_value)

In [139]:
user_items_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87625 entries, 0 to 87624
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           87625 non-null  object
 1   steam_id          87625 non-null  int64 
 2   user_url          87625 non-null  object
 3   items_count       87625 non-null  int64 
 4   item_id           87625 non-null  object
 5   item_name         87625 non-null  object
 6   playtime_forever  87625 non-null  object
 7   playtime_2weeks   87625 non-null  object
dtypes: int64(2), object(6)
memory usage: 5.3+ MB


In [152]:
data_analysis_utils.column_data_types_summary(user_items_2, "item_id")

{'column_name': 'item_id', 'data_type_counts': {"<class 'list'>": 87625}}

In [149]:
print(user_items_2["item_id"][0])
print(type(user_items_2["item_id"][0]))
print(len(user_items_2["item_id"][0]))

print(user_items_2["playtime_forever"][0])
print(type(user_items_2["playtime_forever"][0]))
print(len(user_items_2["playtime_forever"][0]))

[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 3830, 2630, 3900, 34440, 3920, 6400, 6910, 7670, 409710, 220, 320, 340, 360, 380, 400, 420, 9340, 228200, 11450, 7940, 4700, 12900, 13250, 16100, 15700, 15710, 17330, 17340, 22000, 500, 4560, 17460, 10500, 24740, 22200, 26800, 1250, 35420, 3590, 8880, 8890, 23120, 35700, 10140, 35010, 35140, 29180, 15520, 32370, 37700, 6020, 24860, 39530, 550, 223530, 8980, 41500, 20900, 10180, 10190, 17450, 3170, 25900, 31410, 24980, 8850, 409720, 46000, 33230, 20820, 47700, 24960, 43110, 8190, 49600, 31220, 31230, 31240, 31250, 31260, 21090, 21110, 21120, 49400, 33910, 33930, 219540, 17410, 205790, 50130, 62100, 31280, 57300, 21970, 8930, 41000, 41050, 41060, 227780, 18040, 22380, 42700, 42710, 62000, 40800, 9500, 18700, 31270, 58200, 70400, 41300, 41800, 42500, 49900, 55000, 34330, 63200, 99810, 620, 42910, 15500, 15540, 18500, 26500, 35460, 38700, 38720, 63700, 280, 20920, 105600, 98200, 4500, 4540, 4550, 9480, 15620, 20500, 20570, 50620, 55100, 55140, 56

## EDA

### Summary Statistics

### Data Visualization

### Correlation Analysis

### Feature Relationships

### Outlier Detection

### Text Analysis

### Documentation and Reporting

Sentiment analysis of review

# Steam Games

## Importing file

In [96]:
# Importing steam_games.csv file
steam_games_path = "datasets/steam_games.csv"

# Setting low_memory=False - To read the entire file so it determines the data types accurately. 
# Note: It can increase memory usage for larger datasets
steam_games_1 = pd.read_csv(steam_games_path, low_memory=False)

## Data Preparation & Transformation

In [97]:
steam_games_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32134 entries, 0 to 32133
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24063 non-null  object 
 1   genres        28851 non-null  object 
 2   app_name      32132 non-null  object 
 3   title         30084 non-null  object 
 4   url           32134 non-null  object 
 5   release_date  30067 non-null  object 
 6   tags          31971 non-null  object 
 7   reviews_url   32132 non-null  object 
 8   specs         31464 non-null  object 
 9   price         30757 non-null  float64
 10  early_access  32134 non-null  bool   
 11  id            32132 non-null  float64
 12  developer     28835 non-null  object 
dtypes: bool(1), float64(2), object(10)
memory usage: 3.0+ MB


In [98]:
steam_games_2 = steam_games_1.copy()

# Converting values to their repective data types
# "specs" and "genres" have nan values.

# Apply safe_literal_eval to the columns
steam_games_2['genres'] = steam_games_2['genres'].apply(data_analysis_utils.safe_literal_eval)
steam_games_2['specs'] = steam_games_2['specs'].apply(data_analysis_utils.safe_literal_eval)

Games ID have 2 missing values, verifying these values with corresponding game url.<br>

In [99]:
# Iteration of the rows and compare them, if user IDs don't match it will print the index and the value
for index, row in steam_games_2.iterrows():
    id_value = str(row["id"])[:-2]
    url_value = row["url"]
    
    id_len = len(id_value)
    url_id = url_value[34:34+id_len]
    if id_value != url_id:
        print(index, id_value)

74 n
30960 n


If possible, inserting the ids extracted from the url.

In [100]:
# 1st id value
steam_games_2["url"][30960]

'http://store.steampowered.com/app/200260'

In [101]:
# Search for the value in the 'id' column and get the indices
indices = steam_games_2[steam_games_2["id"] == 200260].index
indices

Index([1068], dtype='int64')

The ID already exist

In [102]:
# Deleting corresponding duplicate value
steam_games_2 = steam_games_2.drop(30960)

# Verifying previous action
indices = steam_games_2[steam_games_2["id"] == 30960].index
indices

Index([], dtype='int64')

In [103]:
# 2nd value
steam_games_2["url"][74]

'http://store.steampowered.com/'

In [104]:
# Extract the row using .iloc and print it
value_data = steam_games_2.iloc[74]
print(value_data)

publisher                                  NaN
genres                                     NaN
app_name                                   NaN
title                                      NaN
url             http://store.steampowered.com/
release_date                               NaN
tags                                       NaN
reviews_url                                NaN
specs                                      NaN
price                                    19.99
early_access                             False
id                                         NaN
developer                                  NaN
Name: 74, dtype: object


A lot of information is missing, the value isn't relevant, the value is deleted.

In [153]:
steam_games_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32133
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24062 non-null  object 
 1   genres        28850 non-null  object 
 2   app_name      32131 non-null  object 
 3   title         30083 non-null  object 
 4   url           32132 non-null  object 
 5   release_date  30066 non-null  object 
 6   tags          31970 non-null  object 
 7   reviews_url   32132 non-null  object 
 8   specs         31463 non-null  object 
 9   price         30755 non-null  float64
 10  early_access  32132 non-null  bool   
 11  id            32132 non-null  float64
 12  developer     28834 non-null  object 
dtypes: bool(1), float64(2), object(10)
memory usage: 4.2+ MB


In [105]:
# Deleting corresponding value
steam_games_2 = steam_games_2.drop(74)

# Verifying previous action
indices = steam_games_2[steam_games_2["id"] == 74].index
indices

Index([], dtype='int64')

In [ ]:
data_analysis_utils.column_data_types_summary(user_items_2, "item_id")

In [144]:
steam_games_2["genres"][1000]

['Action', 'Adventure', 'Free to Play', 'Massively Multiplayer', 'RPG']

In [146]:
steam_games_2["specs"][1000]

['Multi-player', 'Co-op', 'Downloadable Content']

## EDA

### Summary Statistics

### Data Visualization

### Correlation Analysis

### Feature Relationships

### Outlier Detection

### Text Analysis

### Documentation and Reporting

Sentiment analysis of review

# Creating files

## API Function 1 "userdata"

Return amount of money spent by the user, the recommendation percentage based on reviews.

### Creating dataframes

Creating new dataframes with only the necessary columns

In [154]:
print(user_reviews_3.columns)
print(user_items_2.columns)
print(steam_games_2.columns)

Index(['user_id', 'user_url', 'item_id', 'posted', 'last_edited', 'helpful',
       'recommend', 'funny', 'review'],
      dtype='object')
Index(['user_id', 'steam_id', 'user_url', 'items_count', 'item_id',
       'item_name', 'playtime_forever', 'playtime_2weeks'],
      dtype='object')
Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')


In [156]:
# Necessary data:

# user_reviews = user_id, user_url, recommend
# user_items = user_url, items_count, item_id
# steam_games = id, price

user_reviews_columns = ["user_id", "item_id", "recommend"]
user_items_columns = ["user_id", "items_count", "item_id", "playtime_forever"] # playtime_forever is added to be used in API function 3
steam_games_columns = ["id", "price"]

# Create new DataFrames with only the selected columns
user_reviews_dataframe_1 = user_reviews_2[user_reviews_columns]
user_items_dataframe_1 = user_items_2[user_items_columns]
steam_games_dataframe_1 = steam_games_2[steam_games_columns]

### Merging dataframes

#### Columns "recommend" and "item_id"

The user_items dataframe have more data than user_reviews.<br>
Identifying missing user_id values from user_reviews in user_items.

In [157]:
# Identify missing user_id values
missing_user_ids = user_reviews_dataframe_1[~user_reviews_dataframe_1["user_id"].isin(user_items_dataframe_1["user_id"])]

# Create a summary of missing user_id values
missing_user_id_summary = missing_user_ids["user_id"]

# Display the missing user_id summary
print("Missing User IDs:")
print(missing_user_id_summary)

Missing User IDs:
1979                  Revan_Aldrei
4044                   BestCyanide
8675           RuslaNLoveStefochka
15513    profiles76561198094860288
Name: user_id, dtype: object


When merging dataframes, this values must be also considered.

##### Renaming the "item_id" column from user_reviews to "item_id_rcmnd"

To avoid conflicts between "item_id" columns from both dataframes, due to both columns represent different content.<br>
- "item_id" from user_reviews, represents the ids of the games reviewed.
- "itme_id" from user_items, represents the ids of all the games owned by the user.


In [158]:
# Proof of difference between values
print(user_items_dataframe_1["item_id"][10000][0])
print(user_reviews_dataframe_1["item_id"][10000][0])

10
215280


In [159]:
# Rename the column 
user_reviews_dataframe_1.rename(columns={"item_id": "item_id_rcmnd"}, inplace=True)

C:\Users\Ronnie\AppData\Local\Temp\ipykernel_11972\3527484091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_reviews_dataframe_1.rename(columns={"item_id": "item_id_rcmnd"}, inplace=True)


In [160]:
user_reviews_dataframe_1

,user_id,item_id_rcmnd,recommend
0,76561197970982479,"[1250, 22200, 43110]","[True, True, True]"
1,js41637,"[251610, 227300, 239030]","[True, True, True]"
2,evcentric,"[248820, 370360, 237930, 263360, 107200, 224500]","[True, True, True, True, True, True]"
3,doctr,"[250320, 20920, 204100, 224600, 207610, 108710]","[True, True, True, True, True, True]"
4,maplemage,"[211420, 211820, 730, 204300]","[True, True, True, True]"
...,...,...,...
25480,76561198306599751,[261030],[True]
25481,Ghoustik,[730],[True]
25482,76561198310819422,[570],[True]
25483,76561198312638244,"[233270, 130, 70, 362890]","[True, True, True, True]"


In [161]:
user_items_dataframe_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87625 entries, 0 to 87624
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           87625 non-null  object
 1   items_count       87625 non-null  int64 
 2   item_id           87625 non-null  object
 3   playtime_forever  87625 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.7+ MB


##### Merging "user_reviews" and "user_items" dataframes

In [162]:
# Merge the two DataFrames using an outer join
users_dataframe_1 = pd.merge(user_items_dataframe_1, user_reviews_dataframe_1, on="user_id", how="outer")
users_dataframe_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87629 entries, 0 to 87628
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           87629 non-null  object 
 1   items_count       87625 non-null  float64
 2   item_id           87625 non-null  object 
 3   playtime_forever  87625 non-null  object 
 4   item_id_rcmnd     25485 non-null  object 
 5   recommend         25485 non-null  object 
dtypes: float64(1), object(5)
memory usage: 4.0+ MB


Verifying all columns have its corresponding values.

In [163]:
# Verifying data types of the "item_id" column
item_id_summary = data_analysis_utils.column_data_types_summary(users_dataframe_1, "item_id")
item_id_summary

{'column_name': 'item_id',
 'data_type_counts': {"<class 'list'>": 87625, "<class 'float'>": 4}}

In [164]:
# Iterating over the column to know indices of float values
for index, row in users_dataframe_1.iterrows():
    value = row["item_id"]
    user_id = row["user_id"]
    items_count = row["items_count"]
    item_id_rcmnd = row["item_id_rcmnd"]
    
    if type(value) == float:
        print(f"index: {index}, user_id: {user_id}, items_count: {items_count}, item_id_rcmnd: {item_id_rcmnd}")

index: 87625, user_id: Revan_Aldrei, items_count: nan, item_id_rcmnd: [233720, 55230, 22380, 313371, 215280, 33100, 250600, 214560]
index: 87626, user_id: BestCyanide, items_count: nan, item_id_rcmnd: [730]
index: 87627, user_id: RuslaNLoveStefochka, items_count: nan, item_id_rcmnd: [730]
index: 87628, user_id: profiles76561198094860288, items_count: nan, item_id_rcmnd: [245550, 206420, 304930, 730, 570]


Manually inserting corresponding values to "items_count" and "item_id"

In [165]:
# Iterating through rows to insert corresponding values
idx_list = [87625, 87626, 87627, 87628]

for idx in idx_list:
    id_list = users_dataframe_1["item_id_rcmnd"][idx]
    list_len = len(id_list)

    users_dataframe_1.loc[idx, "items_count"] = list_len
    # Converting to str type to avoid conflicts
    users_dataframe_1.loc[idx, "item_id"] = str(id_list)

In [166]:
# Applying literal eval to transform to list type, and numeric type
users_dataframe_1['item_id'] = users_dataframe_1['item_id'].apply(data_analysis_utils.safe_literal_eval)
users_dataframe_1['items_count'] = users_dataframe_1['items_count'].apply(data_analysis_utils.safe_literal_eval)

##### Merging values from both lists "item_id" and "item_id_rcmnd"

The values from "item_id" will be the complete list of games owned by the user 

In [167]:
# Calling the function to merge values from lists
data_analysis_utils.merge_values(users_dataframe_1, "item_id", "item_id_rcmnd")

##### Correcting values of "items_count"

In [168]:
# Iterating through the rows of the dataframe to carrect values
for index, row in users_dataframe_1.iterrows():
    count = row["items_count"]
    id_list = row["item_id"]

    list_len = len(id_list)
    
    if count != list_len:
        users_dataframe_1.at[index, "items_count"] = list_len
        

### Generating file

In [120]:
# Necessary data
necessary_columns = ["user_id", "items_count", "item_id", "recommend"]

# Saving the dataframe to a csv format file
# The file will be stored in api inside a the datasets folder 
users_dataframe_1[necessary_columns].to_csv('../api/datasets/user_data_1.csv', index=False)
steam_games_dataframe_1.to_csv('../api/datasets/games_data_1.csv', index=False)

compressing files

In [121]:
# # Compress the CSV file using gzip and save it to a compressed file
# with open('../api/datasets/user_data_1.csv', 'rb') as input_file:
#     with gzip.open('../api/datasets/user_data_1.csv.gz', 'wb') as compressed_file:
#         compressed_file.writelines(input_file)

# print("user_data_1 has been compressed and saved to 'user_data_1.csv.gz'.")

In [122]:
# # Compress the CSV file using gzip and save it to a compressed file
# with open('../api/datasets/games_data_1.csv', 'rb') as input_file:
#     with gzip.open('../api/datasets/games_data_1.csv.gz', 'wb') as compressed_file:
#         compressed_file.writelines(input_file)

# print("games_data_1 has been compressed and saved to 'games_data_1.csv.gz'.")

## API Function 2 "countreviews"

Returns the number of users who made reviews between the given dates and their recommendation percentage based on reviews

### Creating dataframes

Creating new dataframes with only the necessary columns

In [123]:
print(user_reviews_3.columns)

Index(['user_id', 'user_url', 'item_id', 'posted', 'last_edited', 'helpful',
       'recommend', 'funny', 'review'],
      dtype='object')


In [124]:
# Necessary data:

# user_reviews = user_id, posted, recommend
user_reviews_columns = ["posted", "recommend"]

# Create new DataFrames with only the selected columns
user_reviews_dataframe_2 = user_reviews_3[user_reviews_columns]

### Generating file

In [125]:
# Saving the dataframe to a csv format file
# The file will be stored in api inside a the datasets folder 
user_reviews_dataframe_2.to_csv('../api/datasets/user_data_2.csv', index=False)

## API Function 3 "genre"

Returns the position in which a genre is found over the ranking of the same analyzed under the PlayTimeForever column.

### Creating dataframes

Creating new dataframes with only the necessary columns

In [180]:
# Calling the dataframe with complete information of users from the API function 1
print(users_dataframe_1.columns)
print(steam_games_2.columns)

Index(['user_id', 'items_count', 'item_id', 'playtime_forever',
       'item_id_rcmnd', 'recommend'],
      dtype='object')
Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')


In [181]:
# Necessary data:
users_columns = ["item_id", "playtime_forever"]
steam_games_columns = ["id", "genres", "tags", "specs"]

# Create new DataFrames with only the selected columns
users_dataframe_3 = users_dataframe_1[users_columns]
steam_games_dataframe_3 = steam_games_2[steam_games_columns]

#### Deleting null values

In [191]:
users_dataframe_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87629 entries, 0 to 87628
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_id           87629 non-null  object
 1   playtime_forever  87625 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


The "playtime_forever" column have 4 null values.<br>
Apart from being few values, it also means that they do not add time to the genres, so it does not affect the elimination of them. 

In [193]:
# Rows with null values in 'playtime_forever'
null_rows = users_dataframe_3[users_dataframe_3['playtime_forever'].isnull()]
null_rows

,item_id,playtime_forever
87625,"[233720, 55230, 22380, 313371, 215280, 33100, ...",NaN
87626,[730],NaN
87627,[730],NaN
87628,"[245550, 206420, 304930, 730, 570]",NaN


In [194]:
# Delete rows with null values in 'playtime_forever'
users_dataframe_3.dropna(subset=['playtime_forever'], inplace=True)

C:\Users\Ronnie\AppData\Local\Temp\ipykernel_11972\350626787.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_dataframe_3.dropna(subset=['playtime_forever'], inplace=True)


In [195]:
users_dataframe_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87625 entries, 0 to 87624
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_id           87625 non-null  object
 1   playtime_forever  87625 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


### Merging steam game columns

In [182]:
steam_games_dataframe_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32133
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      32132 non-null  float64
 1   genres  28850 non-null  object 
 2   tags    31970 non-null  object 
 3   specs   31463 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.2+ MB


"genre" column have missing values.<br>
"tags" and "specs" also have genre list values, this information must be merge.<br>
The values from "genres" will be the complete list of genres of a game.

#### "genres" and "tags"

In [183]:
# Calling the function to merge values from lists
data_analysis_utils.merge_values(steam_games_dataframe_3, "genres", "tags")

#### "genres" and "specs"

In [184]:
# Calling the function to merge values from lists
data_analysis_utils.merge_values(steam_games_dataframe_3, "genres", "specs")

In [185]:
steam_games_dataframe_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32133
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      32132 non-null  float64
 1   genres  32132 non-null  object 
 2   tags    31970 non-null  object 
 3   specs   31463 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.2+ MB


"tags" and "specs" are no longer necessary

In [186]:
# Necessary data:
steam_columns = ["id", "genres"]

# Create new DataFrames with only the selected columns
steam_games_dataframe_3 = steam_games_dataframe_3[steam_columns]

### Genre ranking dataframe

To avoid saturating the genre ranking function, a dataframe with the ranking of genres is created.

In [201]:
def correspond(dataframe, column_name_1, column_name_2):
    key_val = {}
    report = []
    for idx_row, row in dataframe.iterrows():
        id_list = row[column_name_1]
        time_list = row[column_name_2]
        
        if len(id_list) == len(time_list):

            for idx_id, id in enumerate(id_list):

                if id in key_val:
                    key_val[id] += time_list[idx_id]
                else:
                    key_val[id] = time_list[idx_id]

        else:
            report.append(idx_row)
    
    return key_val, report

In [202]:
users_df_explode, users_report = correspond(users_dataframe_3, "item_id", "playtime_forever")

In [206]:
print(users_report)

[4, 9, 11, 16, 18, 22, 24, 26, 36, 38, 45, 47, 66, 72, 78, 81, 84, 85, 90, 92, 96, 99, 102, 109, 112, 117, 121, 130, 131, 138, 139, 141, 149, 156, 161, 164, 167, 168, 171, 173, 175, 179, 182, 189, 194, 196, 197, 198, 199, 201, 207, 211, 213, 216, 222, 237, 240, 242, 243, 244, 245, 246, 253, 264, 265, 267, 278, 287, 289, 295, 296, 297, 298, 301, 314, 329, 331, 333, 335, 347, 352, 356, 357, 377, 382, 384, 390, 396, 404, 407, 417, 420, 422, 431, 435, 441, 444, 445, 446, 456, 463, 464, 469, 483, 485, 492, 494, 497, 502, 503, 528, 529, 530, 536, 545, 547, 555, 558, 573, 586, 591, 593, 597, 599, 607, 608, 613, 618, 636, 640, 641, 645, 647, 652, 655, 658, 661, 663, 679, 695, 703, 706, 708, 711, 723, 726, 732, 733, 742, 746, 749, 752, 755, 756, 760, 761, 762, 773, 784, 797, 799, 801, 810, 815, 816, 818, 834, 837, 844, 848, 850, 864, 874, 879, 882, 883, 907, 910, 911, 925, 935, 946, 958, 960, 963, 969, 975, 976, 978, 985, 990, 991, 995, 998, 1007, 1015, 1024, 1025, 1028, 1029, 1034, 1047, 1053,

In [218]:
print(len(users_dataframe_3["item_id"][112]))
print(users_dataframe_3["item_id"][112])
print(len(users_dataframe_3["playtime_forever"][112]))
print(users_dataframe_3["playtime_forever"][112])

125
[220, 340, 240, 4000, 400, 3020, 17460, 17410, 10150, 24840, 17450, 24960, 8190, 35140, 13560, 13570, 13580, 33220, 33910, 33930, 219540, 45740, 50620, 40800, 620, 20920, 105600, 67370, 22380, 110800, 209100, 204100, 50300, 211420, 17300, 17330, 17340, 108800, 202170, 730, 219150, 205100, 223470, 220240, 4570, 212070, 218230, 230410, 236390, 243870, 209870, 224580, 203160, 216250, 233270, 242760, 237110, 234080, 22320, 22330, 72850, 218620, 206420, 235600, 250320, 250760, 252490, 252950, 107410, 49520, 221910, 221100, 25000, 234190, 268850, 235460, 550, 223530, 7670, 8850, 8870, 409710, 409720, 214490, 222880, 285900, 287700, 233290, 201810, 295110, 362300, 433850, 439700, 298600, 286690, 287390, 301520, 304930, 241930, 238320, 327740, 330830, 330840, 274940, 333930, 219740, 322330, 236430, 344760, 346110, 407530, 307780, 366844, 292030, 374320, 377160, 391540, 394970, 234140, 247730, 311210, 244930, 379720, 273350, 399730]
124
[904, 20, 1745, 23441, 258, 37, 85, 1084, 273, 1, 47, 

In [197]:
# Merge users_dataframe_3 and steam_games_dataframe_3
merged_df = users_dataframe_3.explode('playtime_forever').merge(steam_games_dataframe_3, left_on='item_id', right_on='id', how='left')

ValueError: You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat

## API Function 4 "userforgenre"

## API Function 5 "developer"

## API Function 6 "sentiment_analysis"

## API Function 7 ML "game_recommendation"